In [111]:
from pyspark import SparkContext
import numpy as np

In [76]:
# workload 1
#close previous sc and open a new one
sc.stop()
sc = SparkContext(appName = "wK1")
# File -> video_id,trending_date,category,views,likes,dislikes,country
file = sc.textFile("AllVideos.csv")
#define a function to get attributions
def getAtrs(row):
    row_list = row.split(",")
    date = row_list[1].split(".")
    seq = (date[0],date[2],date[1])
    row_list[1] = ".".join(seq)
    return ((row_list[0],row_list[6]),(row_list[1],int(row_list[4]),int(row_list[5])))

#define a function to calculate the trending videos of dislike and like
def trend(row):
    key, value = row
    count_dislike = value[-1][2] - value[0][2]
    count_like = value[-1][1] - value[0][1]
    count = count_dislike - count_like
    return key, count

rdd1 = file.map(getAtrs)
rdd2 = rdd1.groupByKey().mapValues(list).filter(lambda x:len(x[1]) > 1)
rdd3 = rdd2.map(trend).sortBy(lambda x: x[1], False).take(10)
for row in rdd3:
    print("‘",row[0][0],"’,",row[1],",‘",row[0][1],"’")

‘ QwZT7T-TXT0 ’, 579119 ,‘ GB ’
‘ QwZT7T-TXT0 ’, 478100 ,‘ US ’
‘ BEePFpC9qG8 ’, 365862 ,‘ DE ’
‘ RmZ3DPJQo2k ’, 334390 ,‘ KR ’
‘ q8v9MvManKE ’, 299044 ,‘ IN ’
‘ pOHQdIDds6s ’, 160365 ,‘ CA ’
‘ ZGEoqPpJQLE ’, 151913 ,‘ RU ’
‘ 84LBjXaeKk4 ’, 134836 ,‘ FR ’
‘ 84LBjXaeKk4 ’, 134834 ,‘ DE ’
‘ 84LBjXaeKk4 ’, 121240 ,‘ RU ’


In [161]:
# workload 2
sc.stop()
sc1.stop()
sc1 = SparkContext(appName="wK2")
file1 = sc1.textFile("AllVideos.csv")
# File -> video_id,trending_date,category,views,likes,dislikes,country
# video_id, category, country are what we need

# define a function to get the attributes we need
def getAtr1(row):
    row_list = row.split(",")
    return ((row_list[0],row_list[2]),row_list[6])

#the country will occur mutiple times, we need unique value for contry
def getUniqueValue(row):
    key = row[0]
    row = list(row[1])
    return key[1],len(np.unique(row))


r1 = file1.map(getAtr1).groupByKey().map(getUniqueValue)
r2 = r1.groupByKey().mapValues(list).mapValues(lambda x:np.average(x)).sortBy(lambda x:x[1])
result = r2.collect()
for row in result:
    print(row)

('Trailers', 1.0)
('Autos & Vehicles', 1.0190448285965426)
('News & Politics', 1.0527098256521152)
('Nonprofits & Activism', 1.057344064386318)
('Education', 1.0628976994615762)
('People & Blogs', 1.063884131133748)
('Pets & Animals', 1.0703560703560704)
('Howto & Style', 1.0875230863944183)
('Travel & Events', 1.0929411764705883)
('Gaming', 1.09443748882132)
('Sports', 1.1421507122296848)
('Entertainment', 1.1446024282935856)
('Science & Technology', 1.1626835588828102)
('Film & Animation', 1.1677314564158094)
('Comedy', 1.2142258635136394)
('Movies', 1.25)
('Music', 1.3105183216252136)
('Shows', 1.555045871559633)


[('Trailers', 1.0),
 ('Autos & Vehicles', 1.0190448285965426),
 ('News & Politics', 1.0527098256521152),
 ('Nonprofits & Activism', 1.057344064386318),
 ('Education', 1.0628976994615762),
 ('People & Blogs', 1.063884131133748),
 ('Pets & Animals', 1.0703560703560704),
 ('Howto & Style', 1.0875230863944183),
 ('Travel & Events', 1.0929411764705883),
 ('Gaming', 1.09443748882132),
 ('Sports', 1.1421507122296848),
 ('Entertainment', 1.1446024282935856),
 ('Science & Technology', 1.1626835588828102),
 ('Film & Animation', 1.1677314564158094),
 ('Comedy', 1.2142258635136394),
 ('Movies', 1.25),
 ('Music', 1.3105183216252136),
 ('Shows', 1.555045871559633)]